In [1]:
import torch
import os
import matplotlib.pyplot as plt
from torchvision import transforms

from Utils.arch import *
from Utils.helper import *
from Utils.loss import *
from Utils.args import *

In [2]:
device = torch.device('cuda:0')
net = Network(device)

## If want to continue from checkpoint
# net.load_state_dict(torch.load('checkpoints_cosine/snn_model_4120.pth'))

net.to(device)
pytorch_total_params = [p.numel() for p in net.parameters() if p.requires_grad]
print(pytorch_total_params)
print(sum(pytorch_total_params))

[9408, 64, 64, 36864, 64, 64, 36864, 64, 64, 36864, 64, 64, 36864, 64, 64, 73728, 128, 128, 147456, 128, 128, 8192, 128, 128, 147456, 128, 128, 147456, 128, 128, 294912, 256, 256, 589824, 256, 256, 32768, 256, 256, 589824, 256, 256, 589824, 256, 256, 1179648, 512, 512, 2359296, 512, 512, 131072, 512, 512, 2359296, 512, 512, 2359296, 512, 512, 65536, 128, 128, 128, 49152, 384, 16384, 128, 65536, 512, 65536, 128, 128, 128, 128, 128, 49152, 384, 16384, 128, 65536, 512, 65536, 128, 128, 128, 128, 128, 49152, 384, 16384, 128, 65536, 512, 65536, 128, 128, 128, 128, 128, 2097152, 128]
13934528


In [3]:
trf =[
    transforms.ColorJitter(0.85,0.85,0.85,0.5),
    # transforms.RandomAffine(30),
    # transforms.RandomPerspective(),
    transforms.RandomRotation(5),
    transforms.GaussianBlur(7),
    # transforms.RandomErasing(p=1,scale=(0.09,0.18),value='random')
    # transforms.RandomAdjustSharpness(0.5),
    # transforms.RandomAutocontrast()
]
T = transforms.RandomChoice(trf)

In [4]:
loss_l=[]
avg_Sp_l=[]
avg_Sn_l=[]
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)

if not os.path.exists(checkpoint_fol):
    os.mkdir(checkpoint_fol)

data_dict,length_dict=create_dict(data_folder)

In [5]:

for it in range(iters):

    batch,Y = construct_batch(P,K,length_dict)

    lis_np = parse_batch(batch,data_dict,img_size)  
    inp = torch.from_numpy(lis_np)
    inp = inp.to(device)
    inp = inp.permute(0, 3, 1, 2)
      
    
    inp = torch.stack([T(x) for x in inp])
    inp = inp.float()
    

    vecs = net(inp)[0]

    loss,avg_Sp,avg_Sn = circle_loss(vecs, Y, gamma, margin, device)
    
    loss = torch.mean(loss)

    loss_l.append(loss.item())
    avg_Sp_l.append(avg_Sp)
    avg_Sn_l.append(avg_Sn)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(str(it+1),f'Train Loss: {loss.item():.4f}')

    plt.figure(figsize=(10, 7))
    plt.plot(
        loss_l, color='green', linestyle='-', 
        label='Loss_graph'
    )
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig("loss_plot.png")
    plt.close('all')

    plt.figure(figsize=(10, 7))
    plt.plot(
        avg_Sn_l, color='red', linestyle='-', 
        label='Sn'
    )
    plt.plot(
        avg_Sp_l, color='green', linestyle='-', 
        label='Sp'
    )
    plt.xlabel('Iterations')
    plt.ylabel('Value')
    plt.legend()
    plt.savefig("Sp-Sn_plot.png")
    plt.close('all')
    
    
    if it%20 ==0:
        torch.save(net.state_dict(), checkpoint_fol+'/model_'+str(it)+'.pth')


torch.save(net.state_dict(), checkpoint_fol+'/model_'+str(it)+'.pth')

1 Train Loss: 163.5358
2 Train Loss: 149.2604
3 Train Loss: 137.9210
4 Train Loss: 128.7604
5 Train Loss: 124.7398
6 Train Loss: 103.6009
7 Train Loss: 85.0506
8 Train Loss: 82.8821
9 Train Loss: 57.2723
10 Train Loss: 52.0194
11 Train Loss: 50.3574
12 Train Loss: 25.7241
13 Train Loss: 18.8425
